In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:
import pandas as pd
from data.random_data import RandomOHLCV
# from data.ohlcv import ServeNewOHLCV
import strategies.ta as ta
import strategies.signals as sig
from frame import Frame

periods = 365

m1 = RandomOHLCV( 
    freq      = '1 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-1, 1),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

m5  = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe()
spy = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing 
etf = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing
day = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing
hr1 = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing




f = Frame('TSLA', run_ta_on_load=True)
f.load_ohlcv(m1)
f.import_data(m5, ['close'], prefix='SPY_')
f.add_ta(ta.MA('SPY_close', 5), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1)

f.add_ta(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='')
# f.add_ta(ta.MA('close', 9),  {'dash': 'solid', 'color': 'magenta', 'width': 2}) # appends ta to the ta list
# f.add_ta(ta.MA('close', 21), {'dash': 'solid', 'color': 'cyan', 'width': 1})
f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1)
f.add_ta(ta.MA('close', 100), {'dash': 'solid', 'color': 'cyan', 'width': 3}, row=1)
f.add_ta(ta.MA('close', 150), {'dash': 'solid', 'color': 'yellow', 'width': 5}, row=1)
f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2)
# f.add_ta(ta.MACD(fast=12, slow=26, signal=9) , 
#          [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
#           {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
#           {'color': 'black'}], 'macd', 2) # histogram

f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high points
          {'color': 'red', 'size': 10}], # low points
          chart_type = 'points')

f.add_ta(ta.MicroTrendline(name='MTREND', pointsCol='HP_hi_10', atrCol='ATR_50', slopeDir='down', slopeToleranceATR=2, projectionPeriod=10),
            {'dash': 'solid', 'color': 'rgb(173, 90, 255)', 'width': 1}, 
            chart_type = 'trendlines')

f.add_ta(ta.SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', atr_col='ATR_50', tolerance=1),
            [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'support_resistance') 

f.add_ta(ta.ConsolidationZone(hp_column='HP_hi_10', lp_column='LP_lo_10', atr_column='ATR_50', price_tolerance=0.001, max_points_between=1, height_width_ratio=50, limit_zones=2),
            [{'color': "rgba(225, 182, 30, 0.5)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'color': "rgba(225, 182, 30, 0.1)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'cons')

# f.setup_chart() 

# f.backtest_setup(
#     start=-10,  # This will slice the data from 0:400
#     end=-1,    # We'll test up to row 450
#     save_snapshots=True
# )

# f.backtest_run(clear_snapshots=True)

In [2]:
import sys
from pathlib import Path
# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path: sys.path.append(str(ROOT_DIR))

import pandas as pd
import strategies.ta as ta
import strategies.signals as sig
from  strategies.ta import TAData
from frame import Frame
from data.random_data import RandomOHLCV
from dataclasses import dataclass

def require_ta_for_all(f, pointsSpan:int=10, atrSpan:int=50):
    f.add_ta_batch([
        TAData(ta.ATR(span=atrSpan), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type=''),
        TAData(ta.HPLP(hi_col='high', lo_col='low', span=pointsSpan), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points'),
        TAData(ta.SupRes(hi_point_col=f'HP_hi_{pointsSpan}', lo_point_col=f'LP_lo_{pointsSpan}', atr_col=f'ATR_{atrSpan}', tolerance=1),
            [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'support_resistance')
    ])

def import_to_daily_df(f, spy:pd.DataFrame=None, etf:pd.DataFrame=None, RSIRow:int=4):
    if spy is not None:
        f.import_data(spy, has_columns=['close'], prefix='SPY_')
        f.add_ta(ta.MansfieldRSI(close_col='close', market_col='SPY_close', span=100), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=RSIRow)
    if etf is not None:
        f.import_data(etf, has_columns=['close'], prefix='ETF_')
        f.add_ta(ta.MansfieldRSI(close_col='close', market_col='ETF_close', span=100), {'dash': 'solid', 'color': 'magenta', 'width': 1}, row=RSIRow)

def import_to_minute_df(f, daily:pd.DataFrame=None, hr4:pd.DataFrame=None, hr1:pd.DataFrame=None):
    if daily is not None: f.import_data(daily, columns_contain=['Sup', 'Res'], prefix='DAILY_' )
    if hr4   is not None: f.import_data(hr4,   columns_contain=['Sup', 'Res'], prefix='HR4_' )
    if hr1   is not None: f.import_data(hr1,   columns_contain=['Sup', 'Res'], prefix='HR1_' )

def ma_ta(f, periods: list[int], ma_col: str = 'close', row: int = 1):
    ma_colour_map = {
        200: {'colour': 'darkslateblue', 'size': 5},
        150: {'colour': 'cornflowerblue', 'size': 4},
        50: {'colour': 'darkmagenta', 'size': 3},
        21: {'colour': 'hotpink', 'size': 2},
        13: {'colour': 'deepskyblue', 'size': 1},
        8: {'colour': 'khaki', 'size': 1}}
    # Create the batch list with the correct color and size
    batch_list = [
        TAData(ta.MA(ma_col, period), {'dash': 'solid', 'color': ma_colour_map[period]['colour'], 'width': ma_colour_map[period]['size']}, row=row)
        for period in periods if period in ma_colour_map
    ]
    
    # Add the batch list to the technical analysis
    f.add_ta_batch(batch_list)

def gaps_ta(f, ls:str='LONG', pointCol:str='HP_hi_10', atrCol:str='ATR_50', lookBack:int=1, row:int=4, chartType:str='lines+markers'):
    l_or_s = ls[0]
    f.add_ta_batch([
        TAData(sig.IsGappedOverPivot(ls=ls, normRange=(0,1), pointCol=pointCol, lookBack=lookBack), {'dash': 'solid', 'color': 'blue', 'width': 1}, chart_type=chartType, row=row),
        TAData(sig.GappedPivots(ls=ls, normRange=(0, 3), pointCol=pointCol, span=400, lookBack=lookBack), {'dash': 'solid', 'color': 'orange', 'width': 1}, chart_type=chartType, row=row),
        TAData(sig.GappedRetracement(ls=ls, normRange=(0,100), pointCol=pointCol, atrCol=atrCol, lookBack=lookBack), {'dash': 'solid', 'color': 'magenta', 'width': 1}, chart_type=chartType, row=row),
        TAData(sig.GappedPastPivot(ls=ls, normRange=(0,100), atrCol=atrCol, pointCol=pointCol, lookBack=lookBack, maxAtrMultiple=10), {'dash': 'solid', 'color': 'red', 'width': 1}, chart_type=chartType, row=row),
        TAData(sig.GapSize(ls=ls, normRange=(0,300), pointCol=pointCol, atrCol=atrCol, lookBack=lookBack), {'dash': 'solid', 'color': 'red', 'width': 1}, chart_type=chartType, row=row),
        TAData(sig.Score(name=f'{ls}_Gaps', normRange=(0,100), lookBack=lookBack, cols=[f'Sig{l_or_s}_GPivs', f'Sig{l_or_s}_GRtc', f'Sig{l_or_s}_GPP', f'Sig{l_or_s}_GSiz'], scoreType='mean', weight=1), {'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type=chartType, row=row)
    ])

def consolidation_ta(f, atrSpan:int=50, consUpperCol:str='CONS_UPPER', consLowerCol:str='CONS_LOWER', maSpan:int=50, lookBack:int=1, colours:list[str]=None, chartType:str='line', scoreRow:int=4):
    colours = ['red', 'green', 'blue', 'yellow', 'magenta', 'cyan', 'orange', 'purple', 'brown', 'pink', 'grey' ] if not colours else colours
    atr_col = f'ATR_{atrSpan}'
    ma_col = f'MA_{maSpan}'
    f.add_ta_batch([
        TAData(ta.ConsolidationZone(hp_column='HP_hi_10', lp_column='LP_lo_10', atr_column='ATR_50', price_tolerance=0.001, max_points_between=1, height_width_ratio=50, limit_zones=2),
            [{'color': "rgba(225, 182, 30, 0.5)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'color': "rgba(225, 182, 30, 0.1)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'cons'),
        TAData(sig.ConsolidationShape(normRange=(1, 20), consUpperCol=consUpperCol, consLowerCol=consLowerCol, atrCol=atr_col, minBars=10, lookBack=lookBack), [{'dash': 'solid', 'color': color, 'width': 2} for color in colours], chart_type=chartType, row=scoreRow),
        TAData(sig.ConsolidationPosition(normRange=(1, 100), consUpperCol=consUpperCol, consLowerCol=consLowerCol, lookBack=lookBack), [{'dash': 'solid', 'color': color, 'width': 2} for color in colours], chart_type=chartType, row=scoreRow),
        TAData(sig.ConsolidationPreMove(normRange=(1, 15), consUpperCol=consUpperCol, consLowerCol=consLowerCol, maCol=ma_col, atrCol=atr_col, lookBack=lookBack), [{'dash': 'solid', 'color': color, 'width': 2} for color in colours], chart_type=chartType, row=scoreRow),
        TAData(sig.Score(rawName='Cons_Score_1', normRange=(0, 100), lookBack=lookBack, containsAllStrings=['Cons', '1'], scoreType='mean', weight=1), {'dash': 'solid', 'color': 'brown', 'width': 4}, chart_type=chartType, row=scoreRow),
        TAData(sig.Score(rawName='Cons_Score_2', normRange=(0, 100), lookBack=lookBack, containsAllStrings=['Cons', '2'], scoreType='mean', weight=1), {'dash': 'solid', 'color': 'darkgreen', 'width': 4}, chart_type=chartType, row=scoreRow)
    ])

def volume_ta(f, ls:str='LONG', ma:int=10, lookBack:int=100, scoreRow:int=3, chartType:str='lines+markers'):
    volMACol = f'MA_vo_{ma}'
    l_or_s = ls[0]
    f.add_ta_batch([
        TAData(ta.MA('volume', ma), {'dash': 'solid', 'color': 'yellow', 'width': 2}, row=2),
        TAData(sig.VolumeSpike(ls=ls, normRange=(0, 200), volMACol=volMACol, lookBack=lookBack), {'dash': 'solid', 'color': 'blue', 'width': 1}, chart_type=chartType, row=scoreRow),
        TAData(sig.VolumeROC(ls=ls, normRange=(0, 300), lookBack=lookBack), {'dash': 'solid', 'color': 'red', 'width': 1}, chart_type=chartType, row=scoreRow),
        TAData(sig.Score(name=f'{l_or_s}_Vol', cols=[f'Sig{l_or_s}_VolSpike', f'Sig{l_or_s}_VolROC'], scoreType='max', weight=1, lookBack=lookBack), {'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type=chartType, row=scoreRow)
    ])



@dataclass
class Event:
    name: str
    valToCehck: str
    checkIf: str
    threshold: str


def STRATEGY_daily_consolidation_bo(f, lookBack:int=100, scoreRow:int=5):
    """Strategies are mainly triggers and validations (YES or NO). for additional nuance use seprate scores"""
    bo = sig.Strategy('BO', lookBack=lookBack)
    bo.add_reset(name='Cl < PPiv', valToCheck='close',  checkIf='<', colThreshold='FFILL_LP_lo_10')
    bo.add_reset(name='Cl < MA150', valToCheck='close', checkIf='<', colThreshold='MA_cl_150')

    bo.add_event     (step=1, name='ConsSco_1 > 50', valToCheck='Cons_Score_1', checkIf='>', colThreshold=50)    
    bo.add_event     (step=1, name='brk Cons',       valToCheck='close',        checkIf='>', colThreshold='CONS_UPPER_1')  
    bo.add_event     (step=1, name='VolSco > 50',    valToCheck='Score_L_Vol',  checkIf='>', colThreshold=50)
    bo.add_validation(step=1, name='Cl > MA50',      valToCheck='close',        checkIf='>', colThreshold='MA_cl_50')
    bo.add_validation(step=1, name='Cl > MA150',     valToCheck='close',        checkIf='>', colThreshold='MA_cl_150')
    bo.add_validation(step=1, name='Cl > Cons1',     valToCheck='close',        checkIf='>', colThreshold='CONS_UPPER_1')

    colours = ['cyan', 'yellow', 'green', 'red', 'orange', 'magenta', 'purple'] * 2
    f.add_ta_batch([
        TAData(ta.Ffill(colToFfill='LP_lo_10'), {}, '', 0),
        TAData(sig.Breaks(price_column='close', direction='above', metric_column='MA_cl_50', normRange=(0,1), lookBack=lookBack), {}, '', 0),
        TAData(bo, [{'dash': 'solid', 'color': color, 'width': 1} for color in colours], chart_type = 'lines+markers', row=scoreRow),
    ])


def STRATEGY_pullback_to_cons(f, lookBack:int=100, scoreRow:int=5):
    """Must happen for a pullback to be valid"""
    bo = sig.Strategy('PB', lookBack=lookBack)
    bo.add_reset(name='Cl < PPiv', valToCheck='close',  checkIf='<', colThreshold='FFILL_LP_lo_10')

    # bo.add_event (step=1, name='LH > 50',      valToCheck='SigL_PB_HLLH',  checkIf='>', colThreshold=50)
    # bo.add_event (step=1, name='ASClr > 50',   valToCheck='SigL_PB_ASClr', checkIf='>', colThreshold=50)
    # bo.add_event (step=1, name='Olap > 50',    valToCheck='SigL_Olap',     checkIf='>', colThreshold=50)
    # bo.add_event (step=1, name='RtcPiv > 50',  valToCheck='SigL_RtcPivs',  checkIf='>', colThreshold=50)
    # bo.add_event (step=1, name='PBnrCons > 50',valToCheck='SigL_PBN_FFI',  checkIf='>', colThreshold=50)

    colours = ['cyan', 'yellow', 'green', 'red', 'orange', 'magenta', 'purple'] * 2

    hp_col_1 = 'HP_hi_10'
    hp_col_2 = 'HP_hi_2'
    lp_col_1 = 'LP_lo_10'
    lp_col_2 = 'LP_lo_2'
    cons_upper_col = 'CONS_UPPER_1'
    atr_col = 'ATR_50'

    f.add_ta_batch([
        TAData(ta.Ffill(colToFfill=lp_col_1), {}, '', 0),
        TAData(ta.Ffill(colToFfill=hp_col_1), {}, '', 0),
        TAData(ta.HPLP(hi_col='high', lo_col='low', span=2), [{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chart_type = 'points'),
        TAData(ta.Ffill(colToFfill=hp_col_2), {}, '', 0),
        TAData(ta.Ffill(colToFfill=cons_upper_col), {}, '', 0),
        TAData(sig.PBPctHLLH      (ls='LONG', normRange=(0,100), lookBack=lookBack, pointCol=hp_col_2, toCol=cons_upper_col, atrCol=atr_col, atrMultiple=1), {'dash': 'solid', 'color': 'cyan', 'width': 1},    row=4, chart_type='line'),
        TAData(sig.PBAllSameColour(ls='LONG', normRange=(0,100), lookBack=lookBack, pointCol=hp_col_2, toCol=cons_upper_col, atrCol=atr_col, atrMultiple=1), {'dash': 'solid', 'color': 'magenta', 'width': 1}, row=4, chart_type='line'),
        TAData(sig.PBOverlap      (ls='LONG', normRange=(0,100), lookBack=lookBack, pointCol=hp_col_2, toCol=cons_upper_col, atrCol=atr_col, atrMultiple=1), {'dash': 'solid', 'color': 'yellow', 'width': 1},  row=4, chart_type='line'),
        # TAData(sig.Trace(name='RtcPivs', ls='LONG', normRange=(0,100), lookBack=lookBack, fromCol='FFILL_HP_hi_10', toCol='FFILL_LP_lo_10', optimalRtc=50), {'dash': 'solid', 'color': 'green', 'width': 1}, row=4, chart_type='line'),
        TAData(sig.PullbackNear(ls='LONG', normRange=(0,100), lookBack=lookBack, fromCol='HP_hi_2', toCol='FFILL_CONS_UPPER_1', optimalRetracement=99, atrCol='ATR_50', atrMultiple=1), {'dash': 'solid', 'color': 'orange', 'width': 1}, row=4, chart_type='line'),
        TAData(bo, [{'dash': 'solid', 'color': color, 'width': 1} for color in colours], chart_type = 'lines+markers', row=scoreRow),
    ])


periods = 400

# ---------------------------------------------
# --------------- DAILY FRAME -----------------
# ---------------------------------------------
spy = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing 
etf = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing
day = RandomOHLCV( freq='5 mins', start='2024',periods= periods, open_rng=(-0.35, 0.4)).get_dataframe() #!supposd to be daily.  just used for testing
f_day = Frame('TSLA', day, run_ta_on_load=True, rowHeights=[0.1, 0.1, 0.1, 0.1, 0.5])
import_to_daily_df(f_day, spy, etf, RSIRow=3) #! test mansfield on real ccharts and compare to trading view
require_ta_for_all(f_day)
ma_ta(f_day, [50, 150, 200])
volume_ta(f_day, ls='LONG', ma=10, scoreRow=4, lookBack=400)
consolidation_ta(f_day, atrSpan=50, maSpan=50, lookBack=400, scoreRow=4)
STRATEGY_daily_consolidation_bo(f_day, lookBack=500, scoreRow=5)
STRATEGY_pullback_to_cons(f_day, lookBack=500, scoreRow=5)
print('--- DAILY FRAME ---')
print(f_day.data.columns)
f_day.plot(height=1000, width=1800)
# display(f_day.data[['close','SigL_PBN_CON', 'FFILL_HP_hi_10']])

# # ---------------------------------------------
# # --------------- HTF FRAMEs -------------------
# hr1 = RandomOHLCV( freq='60 mins', start='2024',periods= periods).get_dataframe() #!supposd to be daily.  just used for testing
# m15 = RandomOHLCV( freq='15 mins', start='2024',periods= periods).get_dataframe()
# f_hr1 = Frame('TSLA', hr1, run_ta_on_load=True)
# f_m15 = Frame('TSLA', m15, run_ta_on_load=True)
# for f in [f_hr1, f_m15]:
#     require_ta_for_all(f)
#     ma_ta(f, [21, 50])
#     print('--- HTF FRAME ---')
#     f.plot()


# # ---------------------------------------------
# # --------------- MINUTE FRAMES ---------------
# # ---------------------------------------------
# m5  = RandomOHLCV( freq='5 mins', start='2024',periods= periods).get_dataframe()
# m2  = RandomOHLCV( freq='2 mins', start='2024',periods= periods).get_dataframe()
# m1  = RandomOHLCV( freq='1 min' , start='2024',periods= periods).get_dataframe()
# f_m5  = Frame('TSLA', m5,  run_ta_on_load=True)
# f_m2  = Frame('TSLA', m2,  run_ta_on_load=True)
# f_m1  = Frame('TSLA', m1,  run_ta_on_load=True)

# for f in [f_m5, f_m2, f_m1]:
#     require_ta_for_all(f)
#     ma_ta(f, [8, 21, 50], ma_col='close', row=1)
#     volume_ta(f, ls='LONG', ma=13, scoreRow=4)
#     consolidation_ta(f, atrSpan=50, maSpan=50, lookBack=1)
#     print('--- MINUTE FRAME ---')
#     f.plot()

    

--- DAILY FRAME ---
Index(['open', 'high', 'low', 'close', 'volume', 'SPY_close', 'MRSI_100_SPY',
       'ETF_close', 'MRSI_100_ETF', 'ATR_50', 'HP_hi_10', 'LP_lo_10', 'Res_1',
       'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower',
       'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper',
       'Sup_2_Lower', 'MA_cl_50', 'MA_cl_150', 'MA_cl_200', 'MA_vo_10',
       'SigL_VolSpike', 'SigL_VolROC', 'Score_L_Vol', 'CONS_UPPER_1',
       'CONS_LOWER_1', 'ConsShape_1', 'ConsPos_1', 'ConsPreMove_1',
       'Cons_Score_1', 'Cons_Score_2', 'FFILL_LP_lo_10',
       'BRK_close_ab_MA_cl_50', 'Stgy_BO_R_Cl < PPiv', 'Stgy_BO_R_Cl < MA150',
       'Stgy_BO_1-ConsSco_1 > 50', 'Stgy_BO_1-brk Cons',
       'Stgy_BO_1-VolSco > 50', 'Stgy_BO_1-Cl > MA50', 'Stgy_BO_1-Cl > MA150',
       'Stgy_BO_1-Cl > Cons1', 'Stgy_BO_step1', 'Stgy_BO_total',
       'FFILL_HP_hi_10', 'HP_hi_2', 'LP_lo_2', 'FFILL_HP_hi_2',
       'FFILL_CONS_UPPER_1', 'SigL_PB_HLLH', 'SigL_PB_ASClr', 'Sig

In [9]:


def get_valid_pb(ls, df, pointCol:str, toCol:str, atrCol:str, minLen:int=3, atrMultiple:int=1):
    # check if names exists in df
    if not all([pointCol in df.columns, toCol in df.columns, atrCol in df.columns]):
        return None
    
    # print(f'{ls} {pointCol=} {toCol=} {atrCol} {minLen} {atrMultiple}')
    # check if has points
    points = df[pointCol].dropna()
    # print(f'length of points: {len(points)}')
    if len(points) < 2:
        return None
    
    # check if window long enough
    w0 = df.loc[points.index[-1]:]
    # print(f'lenght of w0: {len(w0)}')
    if len(w0) < minLen:
        return None
    
    # important to check if the value are ok at the point and not after the point
    toVal = w0[toCol].iat[0]
    atr = w0[atrCol].iat[0]

    # print(f'{toVal=} {atr=}')

    if any([pd.isna(toVal), pd.isna(atr)]):
        return None
    
    if ls == 'LONG':
        hp_bar_low = w0.low.iat[0]
        
        # Check if the distance between high point and toCol is at least n ATRs
        distance = hp_bar_low - toVal
        if distance <= 0:
            return None
        
        if distance < (atrMultiple * atr):
            return None

        # print(f'{atr=} {hp_bar_low=} {toVal=} {distance=} ')
        return w0
    
    if ls == 'SHORT':       
        lp_bar_high = w0.high.iat[0]
        
        # Check if the distance between high point and toCol is at least n ATRs
        distance = toVal - lp_bar_high
        if distance <= 0:
            return None
        
        if distance < (atrMultiple * atr):
            return None
        
        return w0
    
    return None





df = f_day.data.iloc[:-100]
get_valid_pb(ls='SHORT', df=df, pointCol='HP_hi_10', toCol='FFILL_LP_lo_10', atrCol='ATR_50', minLen=3, atrMultiple=1)

In [20]:
df = f_day.data.loc[:'2024-01-02 03:50:00'][['HP_hi_2', 'CONS_UPPER_1', 'ATR_50', 'SigL_PB_HLLH', 'SigL_PB_ASClr', 'SigL_Olap','SigL_PBN_FFI', 'Stgy_PB_R_Cl < PPiv', 'Stgy_PB_total']].tail(50)
display(df)

get_valid_pb(ls='LONG', df=df, pointCol='HP_hi_2', toCol='CONS_UPPER_1', atrCol='ATR_50', minLen=3, atrMultiple=1)

,HP_hi_2,CONS_UPPER_1,ATR_50,SigL_PB_HLLH,SigL_PB_ASClr,SigL_Olap,SigL_PBN_FFI,Stgy_PB_R_Cl < PPiv,Stgy_PB_total
date,,,,,,,,,
2024-01-01 23:45:00,NaN,109.33,0.5568,0.00,0.0,0.00,0.0,0,0
2024-01-01 23:50:00,NaN,109.33,0.5546,0.00,0.0,0.00,0.0,0,0
2024-01-01 23:55:00,108.38,109.33,0.5590,0.00,0.0,0.00,0.0,0,0
2024-01-02 00:00:00,NaN,109.33,0.5630,0.00,0.0,0.00,0.0,0,0
2024-01-02 00:05:00,NaN,109.33,0.5664,0.00,0.0,0.00,0.0,0,0
2024-01-02 00:10:00,NaN,109.33,0.5678,0.00,0.0,0.00,0.0,0,0
2024-01-02 00:15:00,NaN,109.33,0.5660,0.00,0.0,0.00,0.0,0,0
2024-01-02 00:20:00,NaN,109.33,0.5638,0.00,0.0,0.00,0.0,0,0
2024-01-02 00:25:00,NaN,109.33,0.5642,0.00,0.0,0.00,0.0,0,0


In [24]:


def fromVal_isAfter_toVal(ls, df, pointCol:str, toCol:str):
    
    def last_change_index(df, col):
        changes = df[col] != df[col].shift(1)
        last_change_index = changes[::-1].idxmax()
        return last_change_index

    # only want to get values for the direction the trace. 
    # it will return values on the bouce back as well as the pullback if we don't do this
    # get the recnt change in prices of the HPs and LPs
    from_last_change = last_change_index(df, pointCol)
    to_last_change = last_change_index(df, toCol)

    print(f'from_last_change: {from_last_change} > to_last_change:{to_last_change}')

    if from_last_change > to_last_change:
        return True
    return False

df = f_day.data.iloc[:-300]
fromVal_isAfter_toVal('LONG', df, 'HP_hi_10', 'LP_lo_10')

from_last_change: 2024-01-01 08:15:00 > to_last_change:2024-01-01 08:15:00


False

In [9]:
f.data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'SPY_close', 'MA_SP_5',
       'ATR_50', 'MA_cl_50', 'MA_cl_100', 'MA_cl_150', 'MA_vo_10', 'HP_hi_10',
       'LP_lo_10', 'MTREND_DOWN_1', 'MTREND_DOWN_2', 'Res_1', 'Res_1_Upper',
       'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower', 'Sup_1',
       'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower',
       'CONS_UPPER_1', 'CONS_LOWER_1', 'CONS_UPPER_2', 'CONS_LOWER_2',
       'SigL_GapPiv', 'SigL_GPivs', 'SigL_GRtc', 'SigL_GPP', 'SigL_GSiz',
       'Score_LONG_Gaps', 'MA_cl_8', 'MA_cl_21', 'ConsShape_1', 'ConsShape_2',
       'ConsPos_1', 'ConsPos_2', 'ConsPreMove_1', 'ConsPreMove_2',
       'Cons_Score'],
      dtype='object')

In [19]:
f.run_snapshots(0,50, plot=True, display_df=False, sleep_time=0.1, width=2000, height=1500)

# f.get_snapshot(22, plot=True, width=2000, height=1500)

ValueError: No snapshots available. Run backtest with save_snapshots=True first.

In [5]:
f.data# .tail(100)
m5.tail(50)
f.backtest_data.tail(50)

,open,high,low,close,volume,SPY_close,MA_SP_5,ATR_50,MA_cl_50,MA_cl_100,...,Sup_1,Sup_1_Upper,Sup_1_Lower,Sup_2,Sup_2_Upper,Sup_2_Lower,CONS_UPPER_1,CONS_LOWER_1,CONS_UPPER_2,CONS_LOWER_2
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 05:14:00,98.84,99.29,98.75,99.23,56,97.77,97.770,0.5142,99.1120,98.8599,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:15:00,99.56,100.07,99.31,99.79,73,97.85,97.786,0.5200,99.1644,98.8588,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:16:00,99.48,99.60,99.21,99.52,43,97.85,97.802,0.5168,99.2100,98.8602,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:17:00,99.37,99.49,99.32,99.44,42,97.85,97.818,0.5142,99.2466,98.8605,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:18:00,99.72,99.94,99.63,99.78,29,97.85,97.834,0.5100,99.2936,98.8625,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:19:00,99.90,99.99,99.74,99.98,15,97.85,97.850,0.5072,99.3494,98.8722,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:20:00,99.58,99.76,99.13,99.24,17,97.37,97.754,0.5092,99.3790,98.8700,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:21:00,98.89,99.22,98.75,99.02,10,97.37,97.658,0.5116,99.3980,98.8674,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
2024-01-01 05:22:00,98.71,98.73,98.37,98.41,6,97.37,97.562,0.5030,99.3924,98.8619,...,99.05,99.05,98.71,98.47,98.47,98.08,100.62,97.93,NaN,NaN
